In [167]:
from pyqubo import Spin
import numpy as np
import time
import dimod
import neal
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

In [168]:
def parse_dimacs(dimacs_input):
    """
    Takes a text file with DIMACS format and returns the number of variables used and the 
    corresponding clauses.
    """
    clauses = []
    with open(dimacs_input, 'r') as file:
        for line in file:
            if line.startswith('p'):
                num_vars = int(line.split()[2])
                #print("NUM VARS:", num_vars)
            elif not line.startswith(('c', 'p', '\n')):
                clause = [int(x) for x in line.split()[:-1]]
                #print("CLAUSE", line, clause)
                clauses.append(clause)
    return num_vars, clauses

In [169]:
def convert_to_Siddhartha_H(num_vars, clauses):
    """
    Takes the number of variables and the clauses used in a MaxSAT problem and builds the Hamiltonian
    proposed in "MAX 2-SAT with up to 108 qubits" by Siddhartha Santra and Gregory Quiroz.
    """
    # Create variables for each literal
    for i in range(1, num_vars + 1):
        globals()[f"s{i}"] = Spin(f"s{i}")

    # Construct Hamiltonian
    H = 2*len(clauses)
    for clause in clauses:
        for var in clause:
            if var > 0:
                H -= globals()[f's{np.abs(var)}']
            else:
                H += globals()[f's{np.abs(var)}']
        
        if np.prod(clause) < 0:
            H -= globals()[f's{np.abs(clause[0])}']*globals()[f's{np.abs(clause[1])}']
        else:
            H += globals()[f's{np.abs(clause[0])}']*globals()[f's{np.abs(clause[1])}']

    return H*1/4

In [170]:
def count_satisfied_clauses(assignment, clauses):
    """
    Given a solution array, checks how many clauses are satisfied.
    """
    satisfied_count = 0
    for clause in clauses:
        for literal in clause:
            if (literal > 0 and assignment[literal - 1] == 1) or (literal < 0 and assignment[-literal - 1] == 0):
                # At least one literal is satisfied
                satisfied_count += 1
                break
    return satisfied_count

In [155]:
# Read file
dimacs_input = "s2v100c1200-1.cnf"

# Get number of variables and clauses
num_vars, clauses = parse_dimacs(dimacs_input)

# Builds Siddhartha's Hamiltonian
H = convert_to_Siddhartha_H(num_vars, clauses)

# Translates into QUBO model
model = H.compile()
bqm = model.to_bqm()

#### Using ExactSolver
##### (not functioning with so many variables)

In [156]:
# sampler = dimod.ExactSolver()
# sample_set = sampler.sample(bqm)
# print("Using ExactSolver()")
# print(sample_set)

#### Using SimulatedAnnealing

In [173]:
sampler = dimod.SimulatedAnnealingSampler()
start_time = time.time()
sample_set = sampler.sample(bqm, num_reads=100)
end_time = time.time()
print("Using SimulatedAnnlearingSampler()... Time:", round(end_time-start_time, 2), "s.")
print(sample_set)


Using SimulatedAnnlearingSampler()... Time: 64.7 s.
   s1 s10 s100 s11 s12 s13 s14 s15 s16 s17 s18 s19 s2 ... s99 energy num_oc.
0   0   0    1   1   0   1   1   0   1   0   1   1  1 ...   1  469.0       1
1   0   0    1   1   0   1   1   0   1   0   1   1  1 ...   1  469.0       1
2   0   0    1   1   0   1   1   0   1   0   1   1  1 ...   1  469.0       1
3   0   0    1   1   0   1   1   0   1   0   1   1  1 ...   1  469.0       1
4   0   0    1   1   0   1   1   0   1   0   1   1  1 ...   1  469.0       1
5   0   0    1   1   0   1   1   0   1   0   1   1  1 ...   1  469.0       1
6   0   0    1   1   0   1   1   0   1   0   1   1  1 ...   1  469.0       1
7   0   0    1   1   0   1   1   0   1   0   1   1  1 ...   1  469.0       1
8   0   0    1   1   0   1   1   0   1   0   1   1  1 ...   1  469.0       1
9   0   0    1   1   0   1   1   0   1   0   1   1  1 ...   1  469.0       1
10  0   0    1   1   0   1   1   0   1   0   1   1  1 ...   1  469.0       1
11  0   0    1   1   0  

#### Check how many clauses satisfy a given solution

In [174]:
best_sample = sample_set.first.sample
sorted_best_sample = dict(sorted(best_sample.items(), key=lambda item: (len(item[0]), item[0])))
print("Best configuration found:", sorted_best_sample)

Best configuration found: {'s1': 0, 's2': 1, 's3': 1, 's4': 0, 's5': 1, 's6': 1, 's7': 1, 's8': 0, 's9': 1, 's10': 0, 's11': 1, 's12': 0, 's13': 1, 's14': 1, 's15': 0, 's16': 1, 's17': 0, 's18': 1, 's19': 1, 's20': 0, 's21': 0, 's22': 1, 's23': 0, 's24': 0, 's25': 0, 's26': 0, 's27': 1, 's28': 1, 's29': 1, 's30': 0, 's31': 0, 's32': 1, 's33': 0, 's34': 0, 's35': 1, 's36': 1, 's37': 1, 's38': 1, 's39': 1, 's40': 0, 's41': 0, 's42': 0, 's43': 1, 's44': 0, 's45': 0, 's46': 0, 's47': 1, 's48': 0, 's49': 0, 's50': 0, 's51': 1, 's52': 0, 's53': 1, 's54': 1, 's55': 0, 's56': 1, 's57': 1, 's58': 1, 's59': 0, 's60': 0, 's61': 1, 's62': 1, 's63': 1, 's64': 1, 's65': 1, 's66': 0, 's67': 0, 's68': 0, 's69': 0, 's70': 0, 's71': 1, 's72': 1, 's73': 0, 's74': 0, 's75': 1, 's76': 0, 's77': 1, 's78': 1, 's79': 0, 's80': 1, 's81': 0, 's82': 0, 's83': 1, 's84': 0, 's85': 0, 's86': 1, 's87': 1, 's88': 1, 's89': 1, 's90': 0, 's91': 1, 's92': 1, 's93': 1, 's94': 0, 's95': 1, 's96': 1, 's97': 1, 's98': 1, 's

In [175]:
best_configuration_array = np.array([sorted_best_sample[v] for v in sorted_best_sample])
print("Best configuration array:", best_configuration_array)

best_energy = sample_set.first.energy
print("Corresponding energy:", best_energy)

Best configuration array: [0 1 1 0 1 1 1 0 1 0 1 0 1 1 0 1 0 1 1 0 0 1 0 0 0 0 1 1 1 0 0 1 0 0 1 1 1
 1 1 0 0 0 1 0 0 0 1 0 0 0 1 0 1 1 0 1 1 1 0 0 1 1 1 1 1 0 0 0 0 0 1 1 0 0
 1 0 1 1 0 1 0 0 1 0 0 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1]
Corresponding energy: 469.0


In [176]:
satisfied_clauses = count_satisfied_clauses(best_configuration_array, clauses)
print("NUM OF SATISFIED CLAUSES:", satisfied_clauses)

NUM OF SATISFIED CLAUSES: 1031


In [177]:
print("NUM OF UNSATISFIED CLAUSES:", len(clauses) - satisfied_clauses)

NUM OF UNSATISFIED CLAUSES: 169


#### Using DWave QPU
##### Be sure to enter your DWave Leap Token

In [140]:
secure_q = input("Do you really want to use the QPU? (y/n)")
if secure_q == 'y' or secure_q == 'Y':
    q_sampler = EmbeddingComposite(DWaveSampler(token='Your token'))
    q_sample_set = q_sampler.sample(bqm, num_reads=10)
    print("Using DWaveSampler()")
    print(q_sample_set)

Do you really want to use the QPU? (y/n) y


Using DWaveSampler()
  s1 s10 s100 s11 s12 s13 s14 s15 s16 s17 s18 s19 s2 ... s99 energy num_oc. ...
1  1   0    1   1   0   1   1   0   1   0   1   1  1 ...   1  476.0       1 ...
4  0   0    1   1   0   1   1   1   1   0   1   1  1 ...   1  476.0       1 ...
0  1   0    1   1   0   1   1   1   1   0   1   1  1 ...   1  479.0       1 ...
2  0   0    1   1   0   1   1   0   1   0   1   1  1 ...   1  479.0       1 ...
3  1   0    1   1   0   1   1   0   1   0   1   1  1 ...   1  479.0       1 ...
5  0   0    1   1   0   1   1   0   1   0   1   1  1 ...   1  480.0       1 ...
6  1   0    1   1   0   1   1   1   1   0   1   1  1 ...   0  482.0       1 ...
7  1   0    1   1   0   1   1   1   1   0   1   1  1 ...   1  482.0       1 ...
9  0   0    1   1   1   1   1   0   1   0   1   1  1 ...   1  482.0       1 ...
8  1   0    1   1   0   1   1   1   1   0   1   1  1 ...   0  483.0       1 ...
['BINARY', 10 rows, 10 samples, 100 variables]


In [178]:
q_sorted_best_sample = dict(sorted(q_sample_set.first.sample.items(), key=lambda item: (len(item[0]), item[0])))
print("Best configuration found:", q_sorted_best_sample)

Best configuration found: {'s1': 1, 's2': 1, 's3': 1, 's4': 0, 's5': 1, 's6': 1, 's7': 1, 's8': 0, 's9': 1, 's10': 0, 's11': 1, 's12': 0, 's13': 1, 's14': 1, 's15': 0, 's16': 1, 's17': 0, 's18': 1, 's19': 1, 's20': 0, 's21': 0, 's22': 0, 's23': 0, 's24': 0, 's25': 0, 's26': 0, 's27': 1, 's28': 1, 's29': 1, 's30': 0, 's31': 0, 's32': 1, 's33': 0, 's34': 1, 's35': 1, 's36': 0, 's37': 1, 's38': 1, 's39': 1, 's40': 0, 's41': 0, 's42': 0, 's43': 1, 's44': 0, 's45': 0, 's46': 0, 's47': 1, 's48': 0, 's49': 0, 's50': 0, 's51': 0, 's52': 0, 's53': 1, 's54': 1, 's55': 0, 's56': 1, 's57': 1, 's58': 1, 's59': 0, 's60': 0, 's61': 0, 's62': 1, 's63': 1, 's64': 1, 's65': 1, 's66': 0, 's67': 0, 's68': 0, 's69': 0, 's70': 0, 's71': 1, 's72': 1, 's73': 0, 's74': 0, 's75': 1, 's76': 0, 's77': 1, 's78': 1, 's79': 0, 's80': 1, 's81': 0, 's82': 0, 's83': 1, 's84': 0, 's85': 0, 's86': 1, 's87': 1, 's88': 1, 's89': 1, 's90': 0, 's91': 1, 's92': 1, 's93': 1, 's94': 0, 's95': 1, 's96': 0, 's97': 1, 's98': 1, 's

In [179]:
q_best_configuration_array = np.array([q_sorted_best_sample[v] for v in q_sorted_best_sample])
print("Best configuration array:", q_best_configuration_array)

Best configuration array: [1 1 1 0 1 1 1 0 1 0 1 0 1 1 0 1 0 1 1 0 0 0 0 0 0 0 1 1 1 0 0 1 0 1 1 0 1
 1 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 1 1 1 0 0 0 1 1 1 1 0 0 0 0 0 1 1 0 0
 1 0 1 1 0 1 0 0 1 0 0 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1]


In [180]:
q_satisfied_clauses = count_satisfied_clauses(q_best_configuration_array, clauses)
print("NUM OF SATISFIED CLAUSES:", q_satisfied_clauses)

NUM OF SATISFIED CLAUSES: 1024


In [181]:
print("NUM OF UNSATISFIED CLAUSES:", len(clauses) - q_satisfied_clauses)

NUM OF UNSATISFIED CLAUSES: 176
